<a href="https://colab.research.google.com/github/BenRoche18/Im2Calories/blob/master/binary_classification_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds

import os
from matplotlib import pyplot as plt
import numpy as np

TensorFlow 2.x selected.


**Mount google drive**

Mount google drive that contains project files and set to working directory.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = os.path.join("drive", "My Drive", "Im2Calories")
os.chdir(PROJECT_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Enable GPU**



In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Declare parameters**

In [0]:
INPUT_SIZE = 299
SHUFFLE_BUFFER_SIZE = 1000
BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 0.0001

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
loss_func = "binary_crossentropy"

# Input data

**Download 'non-food' dataset**

I will use a downsampled ImageNet dataset for the 'non-food' images since it's free to access and has the broadest variety of images. However the images are limited to a resolution of 64x64 and they aren't labelled, so the 51 food classes will be treated as 'non-food'.

In [0]:
non_food_train_raw = tfds.load("downsampled_imagenet/64x64", split="train")
non_food_val_raw = tfds.load("downsampled_imagenet/64x64", split="validation")

**Download 'food' dataset**

I will the Food-101 dataset for the 'food' images.

In [0]:
food_train_raw = tfds.load("food101", split="train[:80]")
food_val_raw = tfds.load("food101", split="train[80:]")

**Reduce datasets**

As this is only a binary classification, it is unnecessary to use the full datasets and we also need to ensure there are the same number of 'non-food' images as food images.

In [0]:
TRAINING_SIZE = 100000
VAL_SIZE = 10000

food_train_raw = food_train_raw.shuffle(SHUFFLE_BUFFER_SIZE).take(TRAINING_SIZE//2)
non_food_train_raw = non_food_train_raw.shuffle(SHUFFLE_BUFFER_SIZE).take(TRAINING_SIZE//2)
food_val_raw = food_val_raw.shuffle(SHUFFLE_BUFFER_SIZE).take(VAL_SIZE//2)
non_food_val_raw = non_food_val_raw.shuffle(SHUFFLE_BUFFER_SIZE).take(VAL_SIZE//2)

**Format and label the images**

Next, I will format the images for input into a convolutional neural network by rescaling the values between 0 & 1 and resizing to model input size. I will also label the data by labelling food images 1 and non-food images 0.

In [0]:
def formatImage(img):
  img = tf.cast(img, tf.float32)
  img /= 255
  img = tf.image.resize_with_pad(img, INPUT_SIZE, INPUT_SIZE)
  return img

def formatNonFood(features):
  img = features["image"]
  img = formatImage(img)
  return img, 0

def formatFood(features):
  img = features["image"]
  img = formatImage(img)
  return img, 1

food_train_raw = food_train_raw.map(formatFood)
food_val_raw = food_val_raw.map(formatFood)
non_food_train_raw = non_food_train_raw.map(formatNonFood)
non_food_val_raw = non_food_val_raw.map(formatNonFood)

**Merge datasets**

Concatenate the food and 'non-food' datsets.

In [0]:
train_raw = food_train_raw.concatenate(non_food_train_raw)
val_raw = food_val_raw.concatenate(non_food_val_raw)

**Prepare dataset**

Shuffle and batch the images ready for training on the model.

In [0]:
train_batches = train_raw.shuffle(TRAINING_SIZE).batch(BATCH_SIZE, drop_remainder=True)
val_batches = val_raw.batch(BATCH_SIZE, drop_remainder=True) 

**Show example images**

Randomly select and display 20 images from the dataset along with their class (i.e. food or 'non-food').

In [0]:
cols, rows = 4, 5
fig = plt.figure(figsize=(20,10))
fig.suptitle("Random images from dataset")

examples = train_batches.unbatch().batch(20).as_numpy_iterator()

images, labels = next(examples)

for i in range(1, cols*rows+1):
  fig.add_subplot(rows, cols, i)

  img = images[i-1]

  if labels[i-1] == 1:
    plt.title("food")
  else:
    plt.title("non-food")

  plt.axis("off")
  plt.imshow(img, interpolation="nearest")
plt.show()

# Model

In order to train a binary classifier we take a pretrainied Inception model on the ImageNet dataset and replace the classification head with a single logistic node.

In [0]:
base_model = tf.keras.applications.InceptionV3(include_top=False)
base_model.trainable = False

model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss=loss_func, metrics=["accuracy"])

model.summary()

# Training

**Train the classification head**

I now train the classification head of the model on the dataset as the feature extractor part of the model has been frozen.

In [0]:
history = model.fit(train_batches, validation_data=val_batches, epochs=EPOCHS)

**Save model weights**

In [0]:
MODEL_PATH = os.path.join("models", "binary_classifier_v1.h5")
model.save(MODEL_PATH, overwrite=True)

# Evaluate